# Long e Short - Cointegração
- Usaremos Apenas para o Retorno de Pares Cointegrados
- Retorna um CSV - Com os pares cointegrados
- Dados extraindo do API do Yahoo

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from arch.unitroot import ADF

import yfinance as yf 

import warnings
warnings.filterwarnings("ignore")

In [2]:
lista_ativos = list(pd.read_csv('https://raw.githubusercontent.com/Lessalc/Long-Short/main/ativos.txt'))
char_excluido = -3

In [3]:
coluna_df = ['Ativo_Independente', 'Ativo_Dependente', 'ADF-100', 'ADF-120', 'ADF-140', 'ADF-160',
             'ADF-180', 'ADF-200', 'ADF-220', 'ADF-250', 'Total']
df = pd.DataFrame(columns = coluna_df)

In [4]:
dados = yf.download(tickers = lista_ativos, period='2y', interval='1d', auto_adjust=True)

[*********************100%***********************]  83 of 83 completed

1 Failed download:
- UNNAMED: 82: No data found, symbol may be delisted


In [5]:
dados = dados['Close']
dados = dados.loc[dados['^BVSP'].notna(), :]
dados.dropna(axis=1, how='any', inplace=True)
dados.dropna(axis=0, how='all', inplace=True)
dados.drop(columns=['^BVSP'], inplace=True)

In [6]:
dados.shape

(494, 79)

### Atenção - O próximo comando irá demorar um tempo.
- A máquina vai fazer o teste ADF em uma Combinação de 80ativos em 8 diferentes intervalos
- 79 x 78 x 8 = 49296 combinações
- O retorno é o valor ADF

In [7]:
m = 0
for i in range(len(dados.columns)):
    
    ativo_x = dados.columns[i][:char_excluido]
    
    for j in range(len(dados.columns)):
        if j != i:
            ativo_y = dados.columns[j][:char_excluido]
            
            par = dados.iloc[:,[i,j]]
            
            df.loc[m, 'Ativo_Independente'] = ativo_x
            df.loc[m, 'Ativo_Dependente'] = ativo_y
            
            ####################### ESCOLHENDO O INTERVALO ########################
            l_ini = len(par.index)
            intervalo = [100,120,140,160,180,200,220,250]
            k = 2
            soma = 0
            for n in intervalo:
                par_n = par.iloc[l_ini-n:,:]
            
            
                ##############################################################
                
                ###### REGRESSÃO E RESÍDUOS #################################
                X = par_n.iloc[:, 0].values
                y = par_n.iloc[:, 1].values
                X = X.reshape(-1, 1)
                modelo = LinearRegression()
                modelo.fit(X,y)                
                y_pred = modelo.predict(X)                
                residuos = y - y_pred                
                adf = ADF(residuos)
                if adf.stat < adf.critical_values['5%']:
                    df.iloc[m,k] = float(adf.stat)
                    soma = soma + 1
                else:
                    df.iloc[m,k] = 0
                
                k = k+1
            
            df.loc[m,'Total'] = soma
            m = m + 1 

In [8]:
# Teremos o seguinte DataFrame
print(df.shape)
df.head(15)

(6162, 11)


,Ativo_Independente,Ativo_Dependente,ADF-100,ADF-120,ADF-140,ADF-160,ADF-180,ADF-200,ADF-220,ADF-250,Total
0,ABEV3,AZUL4,0,0,0,0,0,0,0,0,0
1,ABEV3,B3SA3,0,0,0,0,0,0,0,0,0
2,ABEV3,BBAS3,0,0,0,0,-3.15917,-2.99946,-3.85922,-3.1283,4
3,ABEV3,BBDC3,0,0,0,-3.04119,-3.26295,-3.90158,-4.32406,-3.57522,5
4,ABEV3,BBDC4,0,-2.94137,-3.14659,-2.99361,-3.35151,-3.36685,-4.76508,-3.05567,7
5,ABEV3,BBSE3,0,0,0,0,-3.17358,-3.06891,-4.01725,0,3
6,ABEV3,BEEF3,0,0,0,0,0,0,0,0,0
7,ABEV3,BPAC11,0,0,0,0,0,0,0,0,0
8,ABEV3,BRAP4,-3.04646,-3.19486,0,0,0,0,0,0,2
9,ABEV3,BRDT3,-3.46952,0,-2.91441,-3.24422,-4.29918,-3.46145,0,0,5


In [9]:
# Podemos agora fazer um subset desse Dataframe
# Quero apenas pares cointegrados em pelo menos 5 períodos
df = df.loc[df['Total'] >= 5, :]
print(df.shape)
df.head(15)

(902, 11)


,Ativo_Independente,Ativo_Dependente,ADF-100,ADF-120,ADF-140,ADF-160,ADF-180,ADF-200,ADF-220,ADF-250,Total
3,ABEV3,BBDC3,0,0,0,-3.04119,-3.26295,-3.90158,-4.32406,-3.57522,5
4,ABEV3,BBDC4,0,-2.94137,-3.14659,-2.99361,-3.35151,-3.36685,-4.76508,-3.05567,7
9,ABEV3,BRDT3,-3.46952,0,-2.91441,-3.24422,-4.29918,-3.46145,0,0,5
16,ABEV3,CMIG4,-3.27543,-3.35973,0,0,-3.2445,-3.26437,-3.60796,-4.23196,6
18,ABEV3,CPFE3,0,-3.24563,-3.04717,-2.89783,-3.13763,-3.77867,0,-3.14954,6
20,ABEV3,CRFB3,-4.03792,-4.02989,-3.87088,-4.11341,-3.11257,0,-2.98737,0,6
26,ABEV3,EGIE3,-3.25426,0,0,-3.19125,-3.83426,-3.34631,-4.08271,-3.73188,6
30,ABEV3,ENBR3,-3.39508,-3.39254,-3.65865,-3.04682,-3.75278,-3.53679,-3.87393,-3.96269,8
32,ABEV3,ENGI11,0,-3.39909,0,-3.0058,-2.96401,-3.44845,-3.68855,-3.16931,6
39,ABEV3,GOLL4,0,0,-2.96477,-3.12518,-3.23374,-2.93816,-3.19422,-2.89867,6


In [10]:
# Por fim salvamos em um arquivo csv. Para usar em outro program
df.to_csv('Pares_Cointegrados - Selecionados.csv', index=False, header=True)